In [1]:
import torch
from models import DCRNN,causal_model,AGCRN,GWNET,STGCN,MTGNN
import tools
from torch.utils.data import DataLoader
from torch.optim import Adam
import numpy as np
from torch import nn
import yaml
import os
import logging
import tools.data_tools
import time
from itertools import product
import csv


import tools.train_tools
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import random
def calculate_coverage(true_values, predicted_means, predicted_lower_bounds, predicted_upper_bounds):
    coverage_rates = []

    for true_val, lower_bound, upper_bound in zip(true_values, predicted_lower_bounds, predicted_upper_bounds):
        # 检查每个真实值是否在预测的上下界内
        coverage = np.logical_and(lower_bound <= true_val, true_val <= upper_bound)

        coverage_rates.append(coverage)

    return coverage_rates
def report_result(coverage, upper_bound, lower_bound, true_values, intere_index,alpha=0.1):
    """
    计算覆盖率、区间长度以及 MIS，并返回结果。
    
    参数:
        coverage (array-like): 预测区间覆盖率的布尔值数组。
        upper_bound (array-like): 预测区间的上限。
        lower_bound (array-like): 预测区间的下限。
        true_values (array-like): 实际值。
        alpha (float): 显著性水平，默认为 0.05。
    
    返回:
        pd.DataFrame: 包含覆盖率、区间长度和 MIS 的数据框。
    """
    result = dict()
    coverage = np.array(coverage)[:,intere_index]
    for i in range(4):
        # 定义区间范围
        if i <= 2:
            range_start = i * (24 * 30)
            range_end = (i + 1) * (24 * 30)
        else:
            range_start = i * (24 * 30)
            range_end = len(coverage)

        # 提取区间对应数据
        coverage_segment = coverage[range_start:range_end]
        upper_bound_segment = upper_bound[range_start:range_end]
        lower_bound_segment = lower_bound[range_start:range_end]
        true_values_segment = true_values[range_start:range_end]

        # 计算覆盖率和区间长度
        result[f'coverage_{i}'] = np.mean(coverage_segment)
        result[f'interval_len_{i}'] = np.mean(upper_bound_segment - lower_bound_segment)
        result[f'minLC_{i}'] = np.mean(coverage_segment,axis=(0,-1)).min()

        # 计算 MIS
        interval_length = upper_bound_segment - lower_bound_segment
        penalty_below = 2 / alpha * (lower_bound_segment - true_values_segment) * (true_values_segment < lower_bound_segment)
        penalty_above = 2 / alpha * (true_values_segment - upper_bound_segment) * (true_values_segment > upper_bound_segment)
        mis = np.mean(interval_length + penalty_below + penalty_above)

        # result[f'MIS_{i}'] = mis

    return pd.DataFrame(result, index=[0])
def predict_with_uncertainty(model, data_loader, device):
    model.eval()
    predictions = []
    true_values = []

    with torch.no_grad():
        for x, y in data_loader:
            x = x.to(device)
            true_values.append(y.cpu().numpy())
            preds = model(x)
            predictions.append(preds.cpu().numpy())

    return np.concatenate(predictions), np.concatenate(true_values)
def compute_conformal_intervals(predictions, residuals, confidence_level=0.9):
    # 计算临界值
    alpha = 1 - confidence_level
    quantile = np.quantile(residuals, confidence_level)
    # print(quantile)
    # print(predictions.shape)
     
    # 构造置信区间
    lower_bounds = predictions - quantile
    upper_bounds = predictions + quantile
    
    return lower_bounds, upper_bounds
def weighted_quantile(values, weights, quantiles, axis=None):
    """
    计算沿指定轴的加权分位数。

    :param values: 数据数组 (多维)
    :param weights: 权重数组 (与values形状一致，或能广播到values形状)
    :param quantiles: 分位数值 (标量或数组，范围 [0, 1])
    :param axis: 沿着哪个轴计算分位数 (默认None，对整个数组计算)
    :return: 加权分位数值 (结果形状取决于axis和quantiles的输入)
    """
    # 转为 numpy 数组
    values = np.array(values)
    weights = np.array(weights)
    quantiles = np.atleast_1d(quantiles)

    if axis is None:
        # 展平处理
        values = values.ravel()
        weights = weights.ravel()
        axis = 0

    # 移动指定轴到最前面
    if axis != 0:
        values = np.moveaxis(values, axis, 0)
        weights = np.moveaxis(weights, axis, 0)

    # 确保 weights 非负且和非零
    if np.any(weights < 0):
        raise ValueError("权重必须为非负值")
    if np.all(weights == 0):
        raise ValueError("权重不能全为0")

    # 获取新轴形状
    shape = values.shape[1:]
    quantile_shape = (len(quantiles),) + shape

    # 输出初始化
    result = np.empty(quantile_shape)

    # 遍历指定轴的切片计算分位数
    for idx in np.ndindex(*shape):
        v_slice = values[(slice(None),) + idx]
        w_slice = weights[(slice(None),) + idx]

        # 排序值和权重
        sorter = np.argsort(v_slice)
        v_sorted = v_slice[sorter]
        w_sorted = w_slice[sorter]

        # 计算累计权重并归一化
        cum_weights = np.cumsum(w_sorted)
        norm_cum_weights = cum_weights / cum_weights[-1]

        # 插值计算分位数
        result[(slice(None),) + idx] = np.interp(quantiles, norm_cum_weights, v_sorted)

    return result
def moving_average(data, window_size):
    # 确保输入是一个 numpy 数组
    data = np.array(data)

    # 如果数据是一维数组，直接进行平滑
    if data.ndim == 1:
        smoothed_data = np.convolve(data, np.ones(window_size) / window_size, mode='same')
    # 如果数据是二维矩阵，逐行进行平滑
    elif data.ndim == 2:
        smoothed_data = np.zeros_like(data)
        for i in range(data.shape[0]):  # 遍历每一行
            smoothed_data[i] = np.convolve(data[i], np.ones(window_size) / window_size, mode='same')
    else:
        raise ValueError("Input data must be 1D or 2D.")

    return smoothed_data

In [ ]:
def get_quantile_inaxis(x, y, axis):
    """
    计算指定轴上的每个切片的分位数。

    参数：
        x (numpy.ndarray): 输入多维数组。
        y (numpy.ndarray): 分位数值数组，长度需与 x 在指定轴上的大小相等。
        axis (int): 指定的轴。

    返回：
        numpy.ndarray: 计算出的分位数数组。
    """
    if x.shape[axis] != len(y):
        raise ValueError("The length of y must be equal to the size of the specified axis in x.")
    
    # 创建一个索引数组
    indices = [slice(None)] * x.ndim
    result = []

    for i, quantile in enumerate(y):
        indices[axis] = i
        this_x = x[tuple(indices)]  # 获取指定轴上的切片
        this_quantile = np.quantile(this_x, quantile)
        result.append(this_quantile)
    
    return np.array(result)


def calculate_conformity_scores(true_vals, q_lo, q_hi):
    """
    计算符合性分数 Ei。
    
    参数:
    - true_vals: NumPy 数组，真实值数组。
    - q_lo: NumPy 数组，下分位数数组。
    - q_hi: NumPy 数组，上分位数数组。
    
    返回:
    - conformity_scores: NumPy 数组，符合性分数数组。
    """
    # 检查输入数组长度是否一致
    if not (len(true_vals) == len(q_lo) == len(q_hi)):
        print(true_vals.shape, q_lo.shape, q_hi.shape)
        raise ValueError("输入数组的长度必须一致")
    
    # 使用矢量化操作计算符合性分数
    conformity_scores = np.where(
        true_vals < q_lo,
        q_lo - true_vals,
        np.where(
            true_vals > q_hi,
            true_vals - q_hi,
            np.maximum(q_lo - true_vals, true_vals - q_hi)
        )
    )
    
    return conformity_scores



def CONTINA(model, val_loader, test_loader, device,config,adeptive_lr=False,alpha_ = 0.001  ):
    
    model.eval()  # 设置模型为评估模式
    true_values = []
    predictions = []
    lower_bounds = []
    upper_bounds = []
    alpha=0.1
    # 第一步：在验证集上计算初始残差集
    use_aci=True
    dynamic_residuals=True
    conformity_scores = []

   

    for inputs, targets in val_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        
        # 模型输出上下分位数
        with torch.no_grad():
            _,q_lo, q_hi = model(inputs)
        
        q_lo = q_lo.cpu().numpy()
        q_hi = q_hi.cpu().numpy()
        true_vals = targets.cpu().numpy()
        
        # 计算符合性分数 Ei
        
        conformity_scores.append(calculate_conformity_scores(true_vals,q_lo,q_hi))
       
    conformity_scores = np.concatenate(conformity_scores)
    initial_scores = conformity_scores.copy()  # 保留初始分数集

    # 第二步：在测试集上生成预测区间
    theta_t=np.array(config['num_nodes']*[0.1])
    theta=[]

 
    beta2 = 0.99   # 二阶动量的衰减率
    epsilon = 1e-8  # 避免除零的小常数

    # 假设 true_val, lower_bound, upper_bound, 和 theta_t 都已经定义
    m_t = np.zeros_like(theta_t)  # 一阶动量
    v_t = np.zeros_like(theta_t)  # 二阶动量
    t = 0  # 时间步
   
    
    with tqdm.tqdm(test_loader, unit="batch", desc=f"{config['dataset_name']}-{model_name}-{lr}") as tepoch:
        for inputs, targets in tepoch:
            theta.append(theta_t.copy())
     
        
            if use_aci:

                q_alpha  = get_quantile_inaxis(conformity_scores,1-theta_t,axis=2) #(114)
                q_alpha=q_alpha.reshape(1,1,config['num_nodes'],1).repeat(2,-1)

            
            inputs, targets = inputs.to(device), targets.to(device)
            
            # 模型输出上下分位数
            with torch.no_grad():
                _,q_lo, q_hi = model(inputs)
            
            q_lo = q_lo.cpu().numpy()
            q_hi = q_hi.cpu().numpy()
            true_val = targets.cpu().numpy()
            
            # 根据 dynamic_residuals 参数选择残差更新模式
            
            conformity_score=calculate_conformity_scores(true_val,q_lo,q_hi)
            
        
            conformity_scores=np.concatenate([conformity_scores,conformity_score])
           
            if dynamic_residuals:
                conformity_scores = conformity_scores[1:]  # 移除最旧分数
        
            lower_bound = q_lo - q_alpha
            upper_bound = q_hi + q_alpha
            prediction = (q_lo + q_hi ) / 2  # 预测均值为中点

            # 保存结果
            true_values.append(true_val)
            predictions.append(prediction)
            lower_bounds.append(lower_bound)
            upper_bounds.append(upper_bound)

            # 如果不使用动态残差，则重置 residuals
            if not dynamic_residuals:
                conformity_scores = initial_scores.copy()
            if use_aci:
                for k in range(len(theta_t)):
                    
                    coverage_errors_this = (true_val[:,:,k,:] < lower_bound[:,:,k,:]) | (true_val[:,:,k,:] > upper_bound[:,:,k,:])
                    err_t = np.mean(coverage_errors_this)
                    
                    # # # 计算梯度（假设是 -err_t，您可以根据实际计算梯度）
                    grad_t = alpha-err_t  # 这里假设 err_t 越小，theta_t 越大，所以梯度是负的
                
                
                
                
                    t += 1  # 更新时间步

                    if adeptive_lr:
                
                        m_t[k] = grad_t
                        v_t[k] = beta2 * v_t[k] + (1 - beta2) * grad_t ** 2
                        
                        # 偏差修正
                        m_hat = m_t[k] / (1)
                        v_hat = v_t[k] / (1 - beta2 ** t)
                        
                        # 更新参数
                        theta_t[k] = theta_t[k] + alpha_ * m_hat / (np.sqrt(v_hat) + epsilon)
                    else:
                        theta_t[k] = alpha_* ( alpha-err_t)+theta_t[k]
                    # 限制 theta_t 的范围
                    theta_t[k] = max(0.01, theta_t[k])
                    theta_t[k] = min(0.99, theta_t[k])
    
    return true_values,predictions,lower_bounds,upper_bounds

# # from matplotlib.ticker import FuncFormatter




In [ ]:
sensitive_res=[]
for lr in [0.005]:
    for dataset_name in ['nycbike']:
        for model_name in ['STGCN']:

            with open(fr'models\{dataset_name}_config.yaml', 'r') as f:
                        config = yaml.safe_load(f)
    
        
        
            config['device']='cuda:0'
            train_data, val_data, test_data, scaler,valid_grid=tools.data_tools.get_datasets(config)
            adj_mx =  np.load(os.path.join('data\\',config['dataset_name'], 'adj_mx.npy'))[valid_grid][:,valid_grid]
            config['num_nodes']=len(valid_grid)
            
            device=torch.device('cuda:0')
        
            if model_name=='DCRNN':
                model = causal_model.CausalModel_quantile_regress(DCRNN.DCRNN,config,adj_mx)
            elif model_name=='MTGNN':
                model = causal_model.CausalModel_quantile_regress(MTGNN.MTGNN,config,adj_mx)
            elif model_name=='GWNET':
                model = causal_model.CausalModel_quantile_regress(GWNET.gwnet,config,adj_mx)
            elif model_name=='STGCN':
                model = causal_model.CausalModel_quantile_regress(STGCN.STGCN,config,adj_mx)
           
    
            
            model=model.to(device)
            train_loader = DataLoader(train_data, batch_size=config['batch_size'], shuffle=True)
            val_loader = DataLoader(val_data, batch_size=config['batch_size'], shuffle=False)
            test_loader = DataLoader(test_data, batch_size=1, shuffle=False)
            model.load_state_dict(torch.load(fr'final_models\{dataset_name}\{model_name}\model.pth'))
            intere_index=np.where(scaler.inverse_transform(train_data.data).mean(axis=1).mean(axis=1)>2)[0]
            true_values, predicted_means, lower_bounds, upper_bounds=CONTINA(model, val_loader, test_loader, device,config,  adeptive_lr=0,alpha_=lr)
            true_values=true_values
            predicted_means=predicted_means
            predicted_lower_bounds=lower_bounds
            predicted_upper_bounds=upper_bounds
            s = calculate_coverage(true_values, predicted_means, predicted_lower_bounds, predicted_upper_bounds)
            np.save(fr'cov\{dataset_name}-{model_name}.npy',s)
            cov_table=report_result(np.array(s)[:,0,0],np.array(predicted_upper_bounds),np.array(predicted_lower_bounds),np.array(true_values)[:,0],intere_index)
            cov_table['dataset_name']=dataset_name
            cov_table['model_name']=model_name
            cov_table['lr']=lr
            sensitive_res.append(cov_table)
            print(cov_table)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_9900\212418416.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(fr'final_models\{dat

   coverage_0  interval_len_0   minLC_0  coverage_1  interval_len_1   minLC_1  \
0    0.899388        0.253525  0.890972    0.900282        0.254412  0.888194   

   coverage_2  interval_len_2   minLC_2  coverage_3  interval_len_3   minLC_3  \
0    0.898921        0.277989  0.888194    0.899541        0.343506  0.887255   

  dataset_name model_name     lr  
0      nycbike      STGCN  0.005  
